# 代码生成器

需求：使用前沿模型从 Python 代码生成高性能的 C++ 代码

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">提醒：执行 C++ 代码是可选的</h2>
            <span style="color:#f71;">作为替代方案，您可以在昨天提供的网站上运行它</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">重要提示</h1>
            <span style="color:#900;">
            在这个实验中，我使用了 Ollama 上的免费开源模型。我也通过 Groq 和 硅基流动 使用了付费的开源模型。请只选择您想要使用的模型！
            </span>
        </td>
    </tr>
</table>

In [2]:
# 导入

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
siliconcloud_api_key = os.getenv('SILICONCLOUD_API_KEY')

if openai_api_key:
    print(f"OpenAI API 密钥存在，开头为 {openai_api_key[:8]}")
else:
    print("未设置 OpenAI API 密钥")
    
if anthropic_api_key:
    print(f"Anthropic API 密钥存在，开头为 {anthropic_api_key[:7]}")
else:
    print("未设置 Anthropic API 密钥（此项为可选）")

if google_api_key:
    print(f"Google API 密钥存在，开头为 {google_api_key[:2]}")
else:
    print("未设置 Google API 密钥（此项为可选）")

if grok_api_key:
    print(f"Grok API 密钥存在，开头为 {grok_api_key[:4]}")
else:
    print("未设置 Grok API 密钥（此项为可选）")

if groq_api_key:
    print(f"Groq API 密钥存在，开头为 {groq_api_key[:4]}")
else:
    print("未设置 Groq API 密钥（此项为可选）")

if openrouter_api_key:
    print(f"OpenRouter API 密钥存在，开头为 {openrouter_api_key[:6]}")
else:
    print("未设置 OpenRouter API 密钥（此项为可选）")

if siliconcloud_api_key:
    print(f"siliconcloud API 密钥存在，开头为 {siliconcloud_api_key[:4]}")
else:
    print("未设置 siliconcloud API 密钥（此项为可选）")

OpenAI API 密钥存在，开头为 sk-proj-
Anthropic API 密钥存在，开头为 sk-ant-
Google API 密钥存在，开头为 AI
Grok API 密钥存在，开头为 xai-
Groq API 密钥存在，开头为 gsk_
未设置 OpenRouter API 密钥（此项为可选）
siliconcloud API 密钥存在，开头为 sk-t


In [4]:
# 连接到客户端库

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"
groq_url = "https://api.groq.com/openai/v1"
ollama_url = "http://localhost:11434/v1"
openrouter_url = "https://openrouter.ai/api/v1"
siliconcloud_url = "https://api.siliconflow.cn/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)
siliconcloud = OpenAI(api_key=siliconcloud_api_key, base_url=siliconcloud_url)

In [5]:
models = ["gpt-5", "claude-sonnet-4-5-20250929", "gemini-2.5-pro", "openai/gpt-oss-120b", "deepseek-ai/DeepSeek-V3.2-Exp", "openai/gpt-oss-20b", "Qwen/Qwen3-Coder-30B-A3B-Instruct", "qwen2.5-coder"]

clients = {"gpt-5": openai, "claude-sonnet-4-5-20250929": anthropic, "gemini-2.5-pro": gemini, "openai/gpt-oss-120b": groq, "deepseek-ai/DeepSeek-V3.2-Exp": siliconcloud, "openai/gpt-oss-20b": groq, "Qwen/Qwen3-Coder-30B-A3B-Instruct": siliconcloud, "qwen2.5-coder": ollama}

In [6]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Darwin',
  'arch': 'arm64',
  'release': '25.1.0',
  'version': 'Darwin Kernel Version 25.1.0: Mon Oct 20 19:34:05 PDT 2025; root:xnu-12377.41.6~2/RELEASE_ARM64_T6041',
  'kernel': '25.1.0',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'arm64-apple-darwin25.1.0'},
 'package_managers': ['xcode-select (CLT)', 'brew'],
 'cpu': {'brand': 'Apple M4 Pro',
  'cores_logical': 14,
  'cores_physical': 14,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'Apple clang version 16.0.0 (clang-1600.0.26.4)',
   'g++': 'Apple clang version 16.0.0 (clang-1600.0.26.4)',
   'clang': 'Apple clang version 16.0.0 (clang-1600.0.26.4)',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': 'GNU Make 3.81'},
  'linkers': {'ld_lld': ''}}}

## 用昨天的命令覆盖此处

或者像昨天一样直接使用网站：

 https://www.programiz.com/cpp-programming/online-compiler/

In [7]:
compile_command = ["clang++", "-std=c++20", "-O3", "-mcpu=native", "-flto=thin", "-DNDEBUG", "main.cpp", "-o", "main"]
run_command = ["./main"]

## 现在，继续主要任务

In [8]:
system_prompt = """
你的任务是将 Python 代码转换为高性能的 C++ 代码。
只用 C++ 代码回应。除了偶尔的注释外，不要提供任何解释。
C++ 的响应需要在尽可能快的时间内产生完全相同的输出。
"""

def user_prompt_for(python):
    return f"""
将此 Python 代码移植到 C++，要求实现尽可能快且产生相同输出。
系统信息是：
{system_info}
你的响应将被写入一个名为 main.cpp 的文件，然后进行编译和执行；编译命令是：
{compile_command}
只用 C++ 代码回应。
要移植的 Python 代码：

```python
{python}
```
"""

In [9]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [10]:
def write_output(cpp):
    with open("main.cpp", "w") as f:
        f.write(cpp)

In [11]:
def port(model, python):
    client = clients[model]
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)
    return reply

In [12]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"结果: {result:.12f}")
print(f"执行时间: {(end_time - start_time):.6f} 秒")
"""

In [13]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"错误: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [14]:
def compile_and_run():
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    except subprocess.CalledProcessError as e:
        print(f"发生错误:\n{e.stderr}")

In [15]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python 代码:", lines=28, value=pi)
        cpp = gr.Textbox(label="C++ 代码:", lines=28)
    with gr.Row():
        model = gr.Dropdown(models, label="选择模型", value=models[0])
        convert = gr.Button("转换代码")

    convert.click(port, inputs=[model, python], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [17]:
compile_and_run()

结果: 3.141592656089
执行时间: 0.240696 秒

结果: 3.141592656089
执行时间: 0.220165 秒

结果: 3.141592656089
执行时间: 0.221795 秒



Qwen 2.5 Coder: 失败  
DeepSeek v3.2-exp: 0.211025  
OpenAI gpt-oss 20B: 0.203833  
Qwen 30B: 0.214230
OpenAI gpt-oss 120B: 0.221795




在 沉默恶魔 的实验中，性能提升如下：

第 8 名：Qwen 2.5 Coder: 失败   
第 7 名：Claude Sonnet 4.5: 48X 速度提升  
第 6 名：OpenAI gpt-oss 120B: 49X 速度提升   
第 5 名：GPT 5 : 50X 速度提升   
第 3 名：DeepSeek v3.2-exp/Qwen 30B: 51X 速度提升
第 2 名：OpenAI gpt-oss 20B: 53X 速度提升
第 1 名：Gemini 2.5 Pro: 81X 速度提升  